In [12]:
import helper
import pickle
import re
import pandas
import numpy as np
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import f1_score
from sklearn.preprocessing import label_binarize

from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

from submission import *

In [2]:
training_data = helper.read_data('./asset/training_data.txt')

def transform_training_data(data):
    after_transformed = []
    non_decimal = re.compile('\d')
    for d in data:
        after_transformed.append(non_decimal.sub('', d))
    return after_transformed
def get_stress(data):
    result = []
    for d in data:
        index = 1
        d = d.split(':')[1]
        for u in d.split(' '):
            if re.match('^[A-Za-z]+[0-9]$', u):
                if u.endswith('1'):
                    result.append(index)
                    break
                index += 1
    return result

training_data = np.array(training_data)

In [6]:
def isConsonant(u):
    consonant = ['P', 'B', 'CH', 'D', 'DH', 'F', 'G', 'HH', 'JH', 'K', 'L', 'M', 'N',
                'NG', 'R', 'S', 'SH', 'T', 'TH', 'V', 'W', 'Y', 'Z', 'ZH']
    try:
        consonant.index(u)
    except:
        return False;
    else:
        return True;

def line_parser(line, isTest):
    data = {}
    data['wd'] = line.split(':')[0]

    ### 特征：字长
    data['len'] = len(data['wd'])

    ### 特征：特殊后缀
    specialSuffix = ['TION', 'BLE', 'ING', 'ED', 'IVE']
    for s in specialSuffix:
        if data['wd'].endswith(s):
            data[s] = 1
    data['ph'] = line.split(':')[1]

    ### 特征：音标数
    data['total_num_ph'] = len(data['ph'].split(' '))

    ### 特征：元音数
    data['v_num'] = 0
    index = 1
    temp = 0
    for u in data['ph'].split(' '):
        if True == isTest:
            if isConsonant(u):
                continue;
            else:
                data['v_num'] += 1
                #data[u] = index
                data[str(index)] = u
                index += 1
        else:
             if re.match('^[A-Za-z]+[0-9]$', u):
                #print(u)
                data['v_num'] += 1
                #data[u[:-1]] = index
                data[str(index)] = u[:-1]
                if u.endswith('1'):
                    temp = index
                index += 1

    ### 特征：每个元音之前的辅音数
    ph = data['ph'].split(' ')
    index = 1
    s = 0
    for i in range(0, len(ph)):
        if isConsonant(ph[i]) == True:
            s += 1
        else:
            data['consonant' + str(index)] = s
            index += 1
            s = 0

    ### 特征：元音距离占比密度

    ### 特征：

    ### 删除辅助属性
    data.pop('ph')
    data.pop('wd')
    if True == isTest:
        return data
    else:
        return data, temp

In [7]:
features = []
for data in training_data:
    features.append(line_parser(data, False))

encoder = LabelEncoder()
vectorizer = DictVectorizer(dtype=int, sparse=False)
x, y = list(zip(*features))
x = vectorizer.fit_transform(x)
y = encoder.fit_transform(y)

In [13]:
param_test1 = {'n_estimators': [50, 60, 70]}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,
                                min_samples_leaf=20,max_depth=8,max_features='sqrt' ,random_state=10), 
                    param_grid = param_test1, scoring='roc_auc',cv=5)
y_post = label_binarize(y, classes=[0, 1, 2, 3])
gsearch1.fit(x,y_post)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.94767, std: 0.00316, params: {'n_estimators': 50},
  mean: 0.94779, std: 0.00331, params: {'n_estimators': 60},
  mean: 0.94787, std: 0.00310, params: {'n_estimators': 70}],
 {'n_estimators': 70},
 0.9478679504707038)